# What is VNC?

VNC is a remote desktop and screen sharing service.

In [ ]:
fluxbox -display :0
x11vnc -forever -nopw -shared -rfbport -ncache 5900 -display :0



Finally, run this middle-ware for the html5 client


In [ ]:
../novnc/utils/launch.sh --vnc localhost:5900


VNC HTML client?

* https://github.com/novnc/noVNC.git


In [17]:
mkdir -p novnc ; if git --work-tree=./novnc branch | grep master; then \
echo "Already checked out novnc"; \
else \
git clone https://github.com/novnc/noVNC.git ./novnc ; \
fi ; ls -la novnc ; pwd


Use vnc with live-server?

In [15]:
npm i -g live-server
live-server novnc


VNC in Docker?

Selenium Docker configured x11vnx and Xvfb for us!


In [ ]:

RUN dos2unix /home/seluser/novnc/utils/launch.sh
RUN sed -i -e 's/export GEOMETRY.*/export GEOMETRY="8160x6120x24"/g' /opt/bin/entry_point.sh
RUN sed -i -e 's/x11vnc/x11vnc -scale 2700x2048 -noxdamage/g' /opt/bin/entry_point.sh
RUN sed -i '/wait \$NODE_PID/i/home/seluser/novnc/utils/launch.sh --vnc localhost:5900 &' /opt/bin/entry_point.sh
USER seluser


In [ ]:
echo "deb http://archive.ubuntu.com/ubuntu xenial main restricted universe multiverse \n" > /etc/apt/sources.list \
  && echo "deb http://archive.ubuntu.com/ubuntu xenial-updates main restricted universe multiverse \n" >> /etc/apt/sources.list \
  && echo "deb http://security.ubuntu.com/ubuntu xenial-security main restricted universe multiverse \n" >> /etc/apt/sources.list \
  && apt-get -qqy update \
  && apt-get -qqy --no-install-recommends install \
    bzip2 \
    ca-certificates \
    openjdk-8-jre-headless \
    tzdata \
    sudo \
    unzip \
    wget \
    x11vnc \
    locales \
    xvfb \
  && echo "UTC" > /etc/timezone \
  && dpkg-reconfigure --frontend noninteractive tzdata \
  && useradd seluser \
         --shell /bin/bash  \
         --create-home \
  && usermod -a -G sudo seluser \
  && echo 'ALL ALL = (ALL) NOPASSWD: ALL' >> /etc/sudoers \
  && echo 'seluser:secret' | chpasswd \
  && sudo mkdir -p /opt/selenium \
  && sudo chown seluser:seluser /opt/selenium \
  && wget --no-verbose https://selenium-release.storage.googleapis.com/3.8/selenium-server-standalone-3.8.1.jar \
    -O /opt/selenium/selenium-server-standalone.jar \
  && wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add - \
  && echo "deb http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list \
  && apt-get update -qqy \
  && apt-get -qqy install \
    google-chrome-stable \
  && CD_VERSION=$(wget -qO- https://chromedriver.storage.googleapis.com/LATEST_RELEASE) \
  && echo "Using chromedriver version: "$CD_VERSION \
  && wget --no-verbose -O /tmp/chromedriver_linux64.zip https://chromedriver.storage.googleapis.com/$CD_VERSION/chromedriver_linux64.zip \
  && rm -rf /opt/selenium/chromedriver \
  && unzip /tmp/chromedriver_linux64.zip -d /opt/selenium \
  && rm /tmp/chromedriver_linux64.zip \
  && mv /opt/selenium/chromedriver /opt/selenium/chromedriver-$CD_VERSION \
  && chmod 755 /opt/selenium/chromedriver-$CD_VERSION \
  && ln -fs /opt/selenium/chromedriver-$CD_VERSION /usr/bin/chromedriver \
  && locale-gen en_US.UTF-8 \
  && dpkg-reconfigure --frontend noninteractive locales \
  && apt-get update -qqy \
  && apt-get -qqy --no-install-recommends install \
    language-pack-en \
    fonts-ipafont-gothic \
    xfonts-100dpi \
    xfonts-75dpi \
    xfonts-cyrillic \
    xfonts-scalable \
    fluxbox \
  && curl -sL https://deb.nodesource.com/setup_9.x | sudo -E bash - \
  && apt-get update -qqy \
  && apt-get -qqy install \
    nodejs \
    build-essential \
  && cd ~ \
  && git clone https://megamindbrian@bitbucket.org/megamindbrian/jupyter_ops.git ./jupytangular \
  && cd ~/jupytangular \
  && npm run install:aws


In [ ]:
vnc entry point?


In [ ]:
#!/bin/bash
#
# IMPORTANT: Change this file only in directory StandaloneDebug!

function shutdown {
  kill -s SIGTERM $NODE_PID
  wait $NODE_PID
}

if [ ! -z "$SE_OPTS" ]; then
  echo "appending selenium options: ${SE_OPTS}"
fi

rm -f /tmp/.X*lock

export DISPLAY=:99 \
  xvfb-run -n 99 --server-args="-screen 0 8160x6120x24 -ac +extension RANDR" \
  java -Dwebdriver.chrome.chromedriver=/opt/selenium/chromedriver-* -jar /opt/selenium/selenium-server-standalone.jar  &
NODE_PID=$!

trap shutdown SIGTERM SIGINT
for i in $(seq 1 10)
do
  xdpyinfo -display $DISPLAY >/dev/null 2>&1
  if [ $? -eq 0 ]; then
    break
  fi
  echo Waiting xvfb...
  sleep 0.5
done

fluxbox -display $DISPLAY &

x11vnc -scale 2700x2048 -noxdamage -forever -nopw -shared -rfbport 5900 -display $DISPLAY &

wait $NODE_PID
